Vector Take #1 Examples

In [1]:
from array import array 
import reprlib
import math

class Vector:
    typecode = 'd'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return f'Vector{components})'

        # Note, this is done to avoid copying every '_components' item to a new list, 
        #    and instead we just remove the beginning and end of the existing array.
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])+ bytes(self._components))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(*self)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

Vector Take #2 Examples

In [3]:
class VectorNumberTwoOne(Vector):
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index):
        return self._components[index]

In [5]:
class MySeq:
    def __getitem__(self, index):
        return index

s = MySeq()
print(s[1])
print(s[1:4])
print(s[1:4:2])
print(s[1:4:2, 9])
print(s[1:4:2, 7:9])


1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [6]:
import operator
class VectorNumberTwoTwo(Vector):
    
    def __len__(self):
        return len(self._components)

    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        index = operator.index(key)
        return self._components[index]

Vector Take #3 Examples

In [ ]:
class VectorNumberThree(VectorNumberTwoTwo):
    __match_args__ = ('x', 'y', 'z', 't')
    
    def __getattr__(self, name):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(name)
        except ValueError:
            pos = -1
        
        if 0 <= pos < len(self._components):
            return self._components[pos]
        msg = f'{cls.__name__!r} object has no attribute {name!r}'
        raise AttributeError(msg)

    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.__match_args__:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

Vector Take #4 Examples

In [9]:
import functools
import operator

n = 0
for i in range(1,6):
    n ^= i
print(n)

print(functools.reduce(lambda a, b: a^b, range(6)))
print(functools.reduce(operator.xor, range(6)))

1
1
1


In [ ]:
from array import array
import reprlib
import math
import functools
import operator

class VectorNumberFour(VectorNumberThree):

    # def __hash__(self):
    #     hashes = (hash(x) for x in self._components)
    #     return functools.reduce(operator.xor, hashes, 0)

    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes)
    
    # def __eq__(self, other):
    #     if len(self) != len(other):
    #         return False
    #     for a, b in zip(self, other):
    #         if a != b:
    #             return False
    #     return True
    def __eq__(self, other):
        return len(self) == len(other) and all(a == b for a, b in zip(self, other))

Vector Take #5 Examples

In [ ]:
import itertools

class VectorNumberFive(VectorNumberFour):
    
    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    
    def angles(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))